In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dot
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Embedding, Dot, Add, Flatten

In [44]:
# Load CSV files
user_data = pd.read_csv("user_data.csv")
mentor_data = pd.read_csv("mentor_data.csv")
content_data = pd.read_csv("content_data.csv")
user_interaction_data = pd.read_csv("user_interaction_data.csv")

In [45]:
# Preprocess the data
user_ids = user_data['user_id'].unique()
mentor_ids = mentor_data['mentor_id'].unique()

user_id_map = {user_id: idx for idx, user_id in enumerate(user_ids)}
mentor_id_map = {mentor_id: idx for idx, mentor_id in enumerate(mentor_ids)}

user_interaction_data['user_idx'] = user_interaction_data['user_id'].map(user_id_map)
user_interaction_data['mentor_idx'] = user_interaction_data['mentor_id'].map(mentor_id_map)

In [46]:
scaler = MinMaxScaler()
y = scaler.fit_transform(user_interaction_data['watch_time'].values.reshape(-1, 1)).flatten()

In [47]:
# Define the model
embedding_dim = 50
user_input = tf.keras.Input(shape=(1,))
mentor_input = tf.keras.Input(shape=(1,))

user_embedding = Embedding(len(user_ids), embedding_dim)(user_input)
mentor_embedding = Embedding(len(mentor_ids), embedding_dim)(mentor_input)

user_bias = Embedding(len(user_ids), 1)(user_input)
mentor_bias = Embedding(len(mentor_ids), 1)(mentor_input)

dot_product = Dot(axes=-1)([user_embedding, mentor_embedding])
sum_biases = Add()([user_bias, mentor_bias])
prediction = Add()([dot_product, sum_biases])
prediction = Flatten()(prediction)

model = Model(inputs=[user_input, mentor_input], outputs=prediction)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [48]:
# Train the model
X = user_interaction_data[['user_idx', 'mentor_idx']].values
model.fit([X[:, 0], X[:, 1]], y, epochs=50, batch_size=64)

Epoch 1/50
157/157 [==============================] - 1s 933us/step - loss: 0.2512
Epoch 2/50
157/157 [==============================] - 0s 878us/step - loss: 0.1087
Epoch 3/50
157/157 [==============================] - 0s 959us/step - loss: 0.0815
Epoch 4/50
157/157 [==============================] - 0s 926us/step - loss: 0.0792
Epoch 5/50
157/157 [==============================] - 0s 949us/step - loss: 0.0766
Epoch 6/50
157/157 [==============================] - 0s 923us/step - loss: 0.0732
Epoch 7/50
157/157 [==============================] - 0s 903us/step - loss: 0.0693
Epoch 8/50
157/157 [==============================] - 0s 891us/step - loss: 0.0657
Epoch 9/50
157/157 [==============================] - 0s 884us/step - loss: 0.0623
Epoch 10/50
157/157 [==============================] - 0s 901us/step - loss: 0.0594
Epoch 11/50
157/157 [==============================] - 0s 889us/step - loss: 0.0569
Epoch 12/50
157/157 [==============================] - 0s 870us/step - loss: 0.0546
E

In [49]:
# Save the model
model.save('mentor_recommender.h5')

In [50]:
# Recommend mentors for a specific user
def recommend_mentors(user_id, top_k=5):
    user_idx = user_id_map[user_id]
    user_vector = model.get_layer(index=2).get_weights()[0][user_idx]
    
    mentor_vectors = model.get_layer(index=3).get_weights()[0]
    similarity_scores = np.dot(mentor_vectors, user_vector)
    
    top_mentor_indices = np.argsort(similarity_scores)[-top_k:][::-1]
    top_mentor_ids = [mentor_ids[idx] for idx in top_mentor_indices]
    
    return top_mentor_ids

In [52]:
# Test the recommender
user_id = 10001
top_5_mentors = recommend_mentors(user_id)
print("Top 5 mentors for user", user_id, ":", top_5_mentors)

Top 5 mentors for user 10001 : [20073, 20100, 20046, 20085, 20028]
